[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1LS4EKO0G8M7bMHqhXKd4R_IkEubBOOUk?authuser=1)


# Muti-Agent Budget Planner

This code implements a Multi-Agent Budget Planner using the CrewAI framework, designed to provide personalized financial planning and advice. The system leverages three specialized AI agents working in concert:

1. An **Information Extractor** that processes and structures raw financial data from user's input
2. A **Financial Analyst** that creates optimized budget plans using the 50/30/20 rule
3. A **Financial Advisor** that provides actionable recommendations

The planner uses `Google's Gemini 1.5 Flash model` as its language model and is built utilizing `CrewAI` framework.

Users can input their financial information - including income, expenses, and savings goals - and receive comprehensive budget analysis and personalized financial advice.

![Muti-Agent Budget Planner System Architecture](https://drive.google.com/uc?export=view&id=1RpeyQ0SnhggVefMN0cle-Dg5ZcbR2vUV)



##01. Envirorment Setup

⚠️ CrewAI requires Python version >=3.10 and <=3.13


In [5]:
import sys
print(sys.version)

3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]


📦 Installing the necessary libraries for the project

In [6]:
!pip install crewai crewai_tools langchain &>/dev/null

## 02. Importing Libraries

In [7]:
from crewai import Agent, Task, Crew, LLM

🔒 Retrieving the Gemini API from the environment variable, follow these step-by-step instructions:

1. Generate the API Key:
   - If you haven’t created an API key yet, visit [aistudio.google.com](https://aistudio.google.com) to generate one.

2. Add the API Key in Google Colab:
   - In your Google Colab notebook, click on the key (secret) icon 🔑 on the left sidebar.
   - Click on the "Add new secret" button.
   - Name the secret `GEMINI_API_KEY`.
   - Paste your generated API key into the Value field.
   - Toggle the Notebook Access.

In [8]:
from google.colab import userdata
GEMINI_API_KEY=userdata.get('GEMINI_API_KEY')

🦜 Initializing the Gemini 1.5 Flash model

In [9]:
gemini_model = LLM (model='gemini/gemini-1.5-flash',
                    temperature=0.7,
                    api_key= GEMINI_API_KEY)

In [10]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

## 03. Creating Agents

In this step, we'll create instances of the `Agent` class, each with specific properties that define their role, goal, and backstory:

- **`role`**: The agent's function within the crew 🛠️
- **`goal`**: The agent's objective, guiding their decision-making 🎯
- **`backstory`**: Provides context to the agent's role and goal, enhancing interactions and collaboration 📖

### Agent 1: Information Extractor 💼📊

In [11]:
Information_Extractor = Agent(
    role="Data Engineer for Financial Systems",

    goal="Process and structure {user_inputs} into"
         "categorized data, identifying gaps or inconsistencies."
         "Ensure accuracy and completeness for subsequent calculations.",

    backstory=(
        "You have a strong background in financial data "
        "processing, having previously worked on systems that categorize and "
        "validate transactions for budgeting apps like Mint. Your expertise "
        "in data pipelines and natural language processing makes you adept at "
        "handling unstructured text and converting it into actionable information."
    ),

    llm = gemini_model,
    allow_delegration=False,
    verbose=True,
    memory=True
)

### Agent 2: Financial Analyst 💰📈

In [12]:
Financial_Analyst = Agent(
    role="Financial Analyst Specializing in Budget Allocation and Optimization.",
    goal="Create a personalized budget plan that aligns with the user's "
         "needs, wants, and financial aspirations."
         "Utilize the 50/30/20 rule as a foundation. ",

    backstory=(
        "With a vast knowledge of financial planning principles, "
        "budgeting methodologies, and investment strategies, you excel at "
        "analyzing complex financial scenarios, identifying potential risks "
        "and rewards, and recommend optimal budget allocations"
        "to help users achieve their financial goals."),

    llm = gemini_model,
    allow_delegration=False,
    verbose=True,
)

### Agent 3: Financial Advisor 💡💵



In [13]:
Financial_Advisor = Agent(
    role="Financial Advisor Specializing in "
         "Personalized Financial Guidance and Education.",

    goal="Empower users to make informed financial choices "
         "and achieve their financial goals by providing them "
         "with actionable financial advice and personalized recommendations",

    backstory=(
        "As a compassionate and knowledgeable financial advisor, you"
        "are passionate about helping others improve their financial well-being. "
        "You possess excellent communication and interpersonal skills, "
        " enabling you to build strong relationships with users "
        "and provide clear, actionable advice."),

    llm = gemini_model,
    allow_delegration=False,
    verbose=True,
)

## 04. Creating Tasks

Now, it's time to define the **tasks**! Tasks are specific assignments for agents and provide all the details they need to execute them successfully.

Each task includes:

- **`description`**: What the task is all about 📝
- **`expected_output`**: What the outcome should look like 🎯
- **`agent`**: The agent responsible for completing the task 🤖

### Task for Information Extractor Agent: Process Financial Data

In [14]:
process_financial_data = Task(
    description=(
        "Process and structure the financial information in {user_inputs}. Categorize data into:"
        "1. Income sources"
        "2. Fixed expenses"
        "3. Discretionary spending"
        "4. Savings goals"
        "Identify any gaps or inconsistencies in the data and request clarification if needed."
    ),

    expected_output=(
        "The output should provide the following structured details:"
        "1. A categorized list of income sources (e.g., salary, freelance, rental income)."
        "2. Fixed expenses categorized by type (e.g., rent, utilities, insurance)."
        "3. Variable expenses categorized by type (e.g., groceries, entertainment)."
        "4. Details of current savings and investments (e.g., bank savings, stocks, bonds)."
        "5. A summary highlighting inconsistencies, missing data, or anomalies in the financial information."
        ),

    agent= Information_Extractor ,

)

### Task for Financial Analyst Agent: Create Budget Plan

- A **list of tasks** can be passed as `context` to another task 🔗
- The task will **consider the outputs** of the tasks in the list during its execution 🛠️
- The task will **not run** until it has received the necessary output(s) from those tasks ⏳

In [15]:
create_budget_plan = Task(
    description=(
        "Create a personalized budget plan based on the processed data. Utilize the 50/30/20 rule:"
        "1. Allocate 50% of income to needs (fixed expenses)"
        "2. Allocate 30% of income to wants (discretionary spending)"
        "3. Allocate 20% of income to savings and debt repayment"
        "Adjust allocations based on the user's specific goals and financial situation."),

    expected_output=(
        "The output should include: "
        "- A detailed budget plan with allocations as per the 50/30/20 rule. "
        "- Needs: 50% of income allocated to fixed expenses. "
        "- Wants: 30% of income allocated to discretionary spending. "
        "- Savings: 20% of income allocated for savings and investments. "
        "- A summary highlighting any adjustments needed."
    ),

    context=[process_financial_data],
    agent= Financial_Analyst
)

### Task for Financial Advisor Agent: Provide Financial Advice

In [16]:
provide_financial_advice = Task(
    description=(
        "Provide actionable financial advice and recommendations. Include"
        "tips on reducing fixed expenses, managing discretionary spending,"
        "achieveing savings goals, and investment strategies for "
        "long-term financial health. Ensure the advice"
        "is personalized and aligns with the user's financial aspirations."
    ),

    expected_output = (
        "Simplified, actionable and personalized advice and steps"
        "for improving financial well-being. Please keep it concise."
        "Provide a structured summary of the 50/30/20 budget "
        "in a simple table format with the following columns:"
        "Category (Needs, Wants, Savings), Amount, and Actionable Steps."
    ),

    context=[create_budget_plan],
    agent= Financial_Advisor
)


## 05. Forming Crew

The **crew** is a group of AI agents working together to complete a set of tasks.

A crew is defined by:

- **`tasks`**: A list of tasks to be performed 📝
- **`agents`**: A list of the agents involved 🤖
- **`verbose=True`**: Allows you to see all execution logs for full transparency 🖥️

In [17]:
budget_planner_crew  = Crew(
    agents=[ Information_Extractor, Financial_Analyst, Financial_Advisor],
    tasks=[ process_financial_data, create_budget_plan, provide_financial_advice],
    verbose=True
)

### Running the crew

In [18]:
user_inputs = """
Each month, I bring in about $6,000, primarily from my salary of $5,000, supplemented by $800 from freelance work and $200 from investments.
My fixed expenses include $1,500 for rent, $200 for utilities, $100 for internet and phone, $400 for insurance, $50 for subscriptions, and $500 for loan payments.
On top of that, I spend around $950 on discretionary expenses with a third for dining out, $150 on hobbies and entertainment, $200 on shopping, and the rest on travel.
I aim to save $1,200 monthly toward a $10,000 goal, which I hope to achieve in about eight months."
"""

### Kicking off the crew

In [19]:
### this execution will take a few minutes to run
result = budget_planner_crew.kickoff(inputs={"user_inputs": user_inputs})

# Agent: Data Engineer for Financial Systems
## Task: Process and structure the financial information in 
Each month, I bring in about $6,000, primarily from my salary of $5,000, supplemented by $800 from freelance work and $200 from investments.
My fixed expenses include $1,500 for rent, $200 for utilities, $100 for internet and phone, $400 for insurance, $50 for subscriptions, and $500 for loan payments.
On top of that, I spend around $950 on discretionary expenses with a third for dining out, $150 on hobbies and entertainment, $200 on shopping, and the rest on travel.
I aim to save $1,200 monthly toward a $10,000 goal, which I hope to achieve in about eight months."
. Categorize data into:1. Income sources2. Fixed expenses3. Discretionary spending4. Savings goalsIdentify any gaps or inconsistencies in the data and request clarification if needed.


# Agent: Data Engineer for Financial Systems
## Final Answer: 
**1. Income Sources:**

*   **Salary:** $5,000
*   **Freelance Work:** $8

## 06. User Input: Enter Your Financial Information 💬💰

Now, please provide your financial details to personalize the process. A budget plan based on your inputs will be created. 📝


In [20]:
# Ask for user input
user_input = input("Please enter your own financial information: ")
test_result = budget_planner_crew.kickoff(inputs={"user_inputs": user_input})

Please enter your own financial information: I earn $3000 monthly, spend $1200 on rent, $300 on utilities, and $600 on hobbies.
# Agent: Data Engineer for Financial Systems
## Task: Process and structure the financial information in I earn $3000 monthly, spend $1200 on rent, $300 on utilities, and $600 on hobbies.. Categorize data into:1. Income sources2. Fixed expenses3. Discretionary spending4. Savings goalsIdentify any gaps or inconsistencies in the data and request clarification if needed.


# Agent: Data Engineer for Financial Systems
## Final Answer: 
**1. Income Sources:**

*   Salary/Employment Income: $3000

**2. Fixed Expenses:**

*   Rent: $1200
*   Utilities: $300

**3. Discretionary Spending:**

*   Hobbies: $600

**4. Savings Goals:**

*   No explicit savings goal is mentioned in the provided data.  This represents a significant gap in the information.  Further clarification is needed to determine savings targets, current savings balances, and investment strategies.  For 